In [127]:
# df_single = df_single_raw.copy()
# df_single["len"] = df_single["smiles"].str.len()
# df_single["features"] = df_single["smiles"].apply(fingerprint_features)
# df_single["features"] = df_single["features"].apply(lambda x: np.array(x))
# df_single["features_one"] = df_single["features"].apply(lambda x: np.sum(np.array(x)==1))
# df_single["features_len"] = df_single["features"].str.len()

In [ ]:
# Tester le downsampling
# pour le deuxième modèle, utiliser les patterns de Géron quand il fait de la prédiction du prochain caractère
# stratify
# learning rate
# focal loss
# class weights
# tf-idf
# smaller batch size

In [154]:
import pandas as pd
import numpy as np
import tensorflow as tf
from feature_extractor import fingerprint_features
import sklearn
from sklearn.model_selection import train_test_split
from tensorflow import keras
import comet_ml

In [155]:
from comet_ml import Experiment

# Create an experiment with your api key
experiment = Experiment(
    api_key="EBS82BdcQJedY2TKiM2v8QhLv",
    project_name="lbc",
    workspace="vinceh91",
)

COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: sklearn, tensorflow, tensorboard. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET WARNING: Failed to check backend version at URL: 'https://www.comet-ml.com/clientlib/isAlive/ver'
COMET ERROR: Run will not be logged 
For more details, please refer to: https://www.comet.ml/docs/python-sdk/warnings-errors/
Traceback (most recent call last):
  File "C:\Users\J0545269\Miniconda3\envs\lbc\lib\site-packages\urllib3\connectionpool.py", line 700, in urlopen
    self._prepare_proxy(conn)
  File "C:\Users\J0545269\Miniconda3\envs\lbc\lib\site-packages\urllib3\connectionpool.py", line 994, in _prepare_proxy
    conn.connect()
  File "C:\Users\J0545

In [2]:
df_single_raw = pd.read_csv("dataset_single.csv")

In [9]:
# Use a utility from sklearn to split and shuffle your dataset.
train_df, test_df = train_test_split(df_single_raw, test_size=0.1)
train_df, val_df = train_test_split(train_df, test_size=0.2)

# Form np arrays of labels and features.
train_labels = np.array(train_df.pop('P1'))
bool_train_labels = train_labels != 0
val_labels = np.array(val_df.pop('P1'))
test_labels = np.array(test_df.pop('P1'))

train_features = np.array(train_df)
val_features = np.array(val_df)
test_features = np.array(test_df)

In [3]:
def oversampling(features: np.ndarray, labels: np.ndarray, oversampling: bool):
    bool_labels = labels != 0
    if oversampling:
        pos_features = features[bool_labels]
        neg_features = features[~bool_labels]
        pos_labels = labels[bool_labels]
        neg_labels = labels[~bool_labels]

        ids = np.arange(len(neg_features))
        choices = np.random.choice(ids, len(pos_features))
        res_neg_features = neg_features[choices]
        res_neg_labels = neg_labels[choices]

        resampled_features = np.concatenate([res_neg_features, pos_features], axis=0)
        resampled_labels = np.concatenate([res_neg_labels, pos_labels], axis=0)

        return resampled_features, resampled_labels
    else:
        return features, labels

    

In [10]:
train_features.shape

(3599, 2)

In [11]:
val_features.shape

(900, 2)

In [12]:
test_features.shape

(500, 2)

In [13]:
resampled_train_features, resampled_train_labels = oversampling(train_features, train_labels, oversampling=True)

In [14]:
resampled_train_features.shape

(5948, 2)

In [15]:
resampled_train_labels.shape

(5948,)

In [16]:
def get_features(arr):
  return np.array([fingerprint_features(s) for s in arr])

In [151]:
extracted_train_features = get_features(resampled_train_features[:,1])
extracted_val_features = get_features(val_features[:,1])
extracted_test_features = get_features(test_features[:,1])

In [ ]:
extracted_train_features.shape


In [150]:
BATCH_SIZE = 16

tf_train_dataset = tf.data.Dataset.from_tensor_slices((extracted_train_features, resampled_train_labels))
tf_val_dataset = tf.data.Dataset.from_tensor_slices((extracted_val_features, val_labels))
tf_test_dataset = tf.data.Dataset.from_tensor_slices((extracted_test_features, test_labels))

tf_train_dataset = tf_train_dataset.shuffle(5000).batch(BATCH_SIZE).prefetch(1)
tf_val_dataset = tf_val_dataset.batch(BATCH_SIZE).prefetch(1)
tf_test_dataset = tf_test_dataset.batch(BATCH_SIZE).prefetch(1)

ValueError: Dimensions 3599 and 5948 are not compatible

In [184]:
for batch in tf_train_dataset:
    break

In [185]:
batch

(<tf.Tensor: shape=(32, 2048), dtype=int32, numpy=
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])>,
 <tf.Tensor: shape=(32,), dtype=int64, numpy=
 array([0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1,
        0, 1, 0, 1, 1, 1, 0, 0, 1, 1], dtype=int64)>)

In [147]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

def make_simple_model(metrics=METRICS, output_bias=None):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)
  model = keras.Sequential([
      keras.layers.Dense(
          2, activation='relu',
          input_shape=(extracted_train_features.shape[-1],)),
      # keras.layers.Dropout(0.5),
      keras.layers.Dense(1, activation='sigmoid',
                         bias_initializer=output_bias),
  ])

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-3),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model

In [22]:
model_over = make_simple_model()

In [23]:
model_over.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 16392     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 16,401
Trainable params: 16,401
Non-trainable params: 0
_________________________________________________________________


In [24]:
history = model_over.fit(tf_train_dataset, validation_data=tf_val_dataset, epochs=15)

Epoch 1/15
186/186 [==============================] - 2s 10ms/step - loss: 0.6697 - tp: 1347.0000 - fp: 691.0000 - tn: 2283.0000 - fn: 1627.0000 - accuracy: 0.6103 - precision: 0.6609 - recall: 0.4529 - auc: 0.6460 - prc: 0.6589 - val_loss: 0.5957 - val_tp: 556.0000 - val_fp: 99.0000 - val_tn: 68.0000 - val_fn: 177.0000 - val_accuracy: 0.6933 - val_precision: 0.8489 - val_recall: 0.7585 - val_auc: 0.6230 - val_prc: 0.8715
Epoch 2/15
186/186 [==============================] - 0s 2ms/step - loss: 0.5107 - tp: 2282.0000 - fp: 665.0000 - tn: 2309.0000 - fn: 692.0000 - accuracy: 0.7719 - precision: 0.7743 - recall: 0.7673 - auc: 0.8470 - prc: 0.8491 - val_loss: 0.6002 - val_tp: 541.0000 - val_fp: 98.0000 - val_tn: 69.0000 - val_fn: 192.0000 - val_accuracy: 0.6778 - val_precision: 0.8466 - val_recall: 0.7381 - val_auc: 0.6281 - val_prc: 0.8791
Epoch 3/15
186/186 [==============================] - 0s 2ms/step - loss: 0.4090 - tp: 2438.0000 - fp: 428.0000 - tn: 2546.0000 - fn: 536.0000 - accur

# Without resampling

In [152]:
extracted_train_features = get_features(train_features[:,1])

tf_train_dataset = tf.data.Dataset.from_tensor_slices((extracted_train_features, train_labels))
tf_val_dataset = tf.data.Dataset.from_tensor_slices((extracted_val_features, val_labels))
tf_test_dataset = tf.data.Dataset.from_tensor_slices((extracted_test_features, test_labels))

tf_train_dataset = tf_train_dataset.shuffle(5000).batch(BATCH_SIZE).prefetch(1)
tf_val_dataset = tf_val_dataset.batch(BATCH_SIZE).prefetch(1)
tf_test_dataset = tf_test_dataset.batch(BATCH_SIZE).prefetch(1)

In [153]:
model = make_simple_model()
history = model.fit(tf_train_dataset, validation_data=tf_val_dataset, epochs=50)

Epoch 1/50
225/225 [==============================] - 2s 8ms/step - loss: 0.5147 - tp: 3331.0000 - fp: 694.0000 - tn: 98.0000 - fn: 376.0000 - accuracy: 0.7622 - precision: 0.8276 - recall: 0.8986 - auc: 0.5336 - prc: 0.8469 - val_loss: 0.4741 - val_tp: 733.0000 - val_fp: 167.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_accuracy: 0.8144 - val_precision: 0.8144 - val_recall: 1.0000 - val_auc: 0.5838 - val_prc: 0.8519
Epoch 2/50
225/225 [==============================] - 1s 3ms/step - loss: 0.4344 - tp: 2974.0000 - fp: 625.0000 - tn: 0.0000e+00 - fn: 0.0000e+00 - accuracy: 0.8263 - precision: 0.8263 - recall: 1.0000 - auc: 0.6859 - prc: 0.9058 - val_loss: 0.4648 - val_tp: 733.0000 - val_fp: 167.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_accuracy: 0.8144 - val_precision: 0.8144 - val_recall: 1.0000 - val_auc: 0.6199 - val_prc: 0.8688
Epoch 3/50
225/225 [==============================] - 1s 2ms/step - loss: 0.4098 - tp: 2970.0000 - fp: 619.0000 - tn: 6.0000 - fn: 4.0000 -

# One batch

In [103]:
for batch in tf_train_dataset:
    break

In [138]:
batch

(<tf.Tensor: shape=(32, 2048), dtype=int32, numpy=
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])>,
 <tf.Tensor: shape=(32,), dtype=int64, numpy=
 array([1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        0, 1, 1, 1, 0, 1, 0, 1, 0, 1], dtype=int64)>)

In [134]:
feat, labs = batch

In [148]:
model_simple = make_simple_model()
model_simple.fit(feat, labs, epochs=100)

Epoch 1/100
1/1 [==============================] - 0s 1ms/step - loss: 0.9494 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 10.0000 - fn: 22.0000 - accuracy: 0.3125 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5455 - prc: 0.6746
Epoch 2/100
1/1 [==============================] - 0s 2ms/step - loss: 0.9386 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 10.0000 - fn: 22.0000 - accuracy: 0.3125 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.6068 - prc: 0.7088
Epoch 3/100
1/1 [==============================] - 0s 999us/step - loss: 0.9284 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 10.0000 - fn: 22.0000 - accuracy: 0.3125 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.6477 - prc: 0.7261
Epoch 4/100
1/1 [==============================] - 0s 2ms/step - loss: 0.9201 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 10.0000 - fn: 22.0000 - accuracy: 0.3125 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.7045 - prc: 0.7883
Epoch 5/100
1/1 [==============================] - 0s 1ms/step - loss:

In [149]:
model_simple.evaluate(tf_val_dataset)

29/29 [==============================] - 0s 2ms/step - loss: 0.5950 - tp: 581.0000 - fp: 123.0000 - tn: 44.0000 - fn: 152.0000 - accuracy: 0.6944 - precision: 0.8253 - recall: 0.7926 - auc: 0.5599 - prc: 0.8417


[0.595022976398468,
 581.0,
 123.0,
 44.0,
 152.0,
 0.6944444179534912,
 0.8252840638160706,
 0.7926329970359802,
 0.5599415302276611,
 0.8416948318481445]